## Set up

In [135]:
from pymongo import MongoClient
from pprint import pprint

import pandas as pd
import re
import nltk

import pickle
import json

In [136]:
# import product list
with open('pickles/product_list.pickle', 'rb') as to_read:
    product_list = pickle.load(to_read)

with open('pickles/review_list.pickle', 'rb') as to_read:
    review_list = pickle.load(to_read)
    
with open('pickles/sim_asin.pickle', 'rb') as to_read:
    sim_asin = pickle.load(to_read)

with open('pickles/all_review_list_SA.pickle', 'rb') as to_read:
    all_review_list = pickle.load(to_read)

In [249]:
all_review_list.head(5)

,asin,title,all_review_text,sentences,compound_scores,pos_sents,pos_scores,neg_sents,neg_scores,topic_3_sent,topic_3_score,topic_4_sent,topic_4_score,topic_5_sent,topic_5_score,topic_6_sent,topic_6_score
0,B0000Y3NO6,"DERMAdoctor Calm, Cool & Corrected anti-rednes...",I gave it a shot for a while until the small b...,[I gave it a shot for a while until the small ...,"[-0.2716, 0.3716, 0.6369, -0.6597, 0.1621, 0.7...","[The cream feels cool on the skin, which is ni...","[0.3716, 0.6369, 0.1621, 0.7506, 0.8242, 0.510...",[I gave it a shot for a while until the small ...,"[-0.2716, -0.6597, -0.6358, -0.2365, 0.0, 0.0,...",[This extremely soothing cream helps any irrit...,0.334000,"[It also dimished my acne outbreaks, which I t...",0.436043,[After trying all the drugstore rosacea produc...,0.577633,"[It also dimished my acne outbreaks, which I t...",0.20230
1,B00012C5RS,"DERMAdoctor Picture Porefect Pore Minimizer, 1...","I haven't splurged on a full-size bottle yet, ...","[I haven't splurged on a full-size bottle yet,...","[0.3612, 0.0, 0.5994, 0.0, 0.0, 0.0, 0.4939, 0...","[I haven't splurged on a full-size bottle yet,...","[0.3612, 0.5994, 0.4939, 0.3384, 0.646, 0.3182...","[I received a sample of this, so the effects I...","[0.0, 0.0, 0.0, 0.0, 0.0, -0.5267, 0.0, -0.153...","[This product really does work, just beware if...",0.318200,[My skin is normally pretty oily and prone to ...,0.359314,[My skin is normally pretty oily and prone to ...,0.340040,[I rant yet again: the packaging: Pretty lav...,0.54540
2,B0001EKTTC,"Glytone Rejuvenating Mask, 3 oz.",Great product . Gets the job done. Leaves your...,"[Great product ., Gets the job done., Leaves y...","[0.6249, 0.0, 0.4173, 0.6194, 0.8555, 0.5574, ...","[Great product ., Leaves your skin absolutely ...","[0.6249, 0.4173, 0.6194, 0.8555, 0.5574, 0.722...","[Gets the job done., This is a keeper!!!!!.., ...","[0.0, 0.0, -0.5267, 0.0, -0.296, 0.0, -0.34, -...",[],NaN,"[Perfect for acne prone skin., I use it once a...",0.594925,"[Very, very good deep cleansing masque, for oi...",0.701500,[],NaN
3,B0001EL5Q8,"PCA SKIN Rejuvenating Serum, 1 fl. oz.",I was very happy with the promptness with whic...,[I was very happy with the promptness with whi...,"[0.8479, 0.0, -0.2481, 0.1511, 0.8934, 0.0, 0....",[I was very happy with the promptness with whi...,"[0.8479, 0.1511, 0.8934, 0.5859, 0.5719, 0.790...",[Time will tell as to whether or not a 70 year...,"[0.0, -0.2481, 0.0, -0.4019, 0.0, 0.0, -0.4019...",[Reduces the signs of aging love itFast delive...,0.674000,[As soon as I stop using the serums my acne co...,0.722700,"[love itLove the way it feels, but not sure if...",0.644250,[I really like the Pca line and notice the dif...,0.52896
4,B0001EL5JA,PCA SKIN Protecting Hydrator Broad Spectrum S...,I bought the Hydrator with SPF after having a ...,[I bought the Hydrator with SPF after having a...,"[0.6249, 0.0, 0.3612, 0.8114, 0.6696, 0.1027, ...",[I bought the Hydrator with SPF after having a...,"[0.6249, 0.3612, 0.8114, 0.6696, 0.1027, 0.440...",[I thought the Hydrator would be a similar tex...,"[0.0, 0.0, -0.2755, 0.0, -0.4003, 0.0, 0.0, -0...",[Highly recommended for sensitive and acne-pro...,0.529167,[Best SPF day moisturizer for oily/acne-prone ...,0.581067,[Best SPF day moisturizer for oily/acne-prone ...,0.631994,[],NaN


In [137]:
# recommend:
# - top 5 products for each category
# - return dictionary with the product info

In [138]:
sim = pd.DataFrame(columns=['prod1','prod2','similarity'])

sim['prod1'] = [i[1][0] for i in sim_asin]
sim['prod2'] = [i[1][1] for i in sim_asin]
sim['similarity'] = [i[0][0][0] for i in sim_asin]

In [139]:
sim.tail(5)

,prod1,prod2,similarity
2509915,B0000Y3NO6,B00AF3RYHE,0.0
2509916,B0000Y3NO6,B008KAA8P4,0.0
2509917,B0000Y3NO6,B005JEK7DC,0.0
2509918,B0000Y3NO6,B003VCKZZO,0.0
2509919,B0000Y3NO6,B000AV31NC,0.0


### generate recommendation

In [2]:
concerns = ['sensitive_skin','acne_control','anti-aging','oil_control','none']

In [266]:
# generate top 5 matches in each category

def get_rec(asin,concern):
    
    def get_other_prod(x):
        if x['prod1'] != asin:
            return x['prod1']
        else:
            return x['prod2']
    
    def get_sims(asin):
        df = pd.DataFrame(sim[(sim['prod1']==asin)|(sim['prod2']==asin)])
        df['selected_asin'] = asin
        df['other_asin'] = df.apply(get_other_prod, axis=1)
        df.drop(['prod1','prod2'],axis=1,inplace=True)
        return df

    # get similarities for that product
    prod = get_sims(asin)
    
    #merge the concern sentiments
    prod2 = pd.merge(prod,all_review_list,left_on='other_asin',right_on='asin',how='inner')
    
    # if concern is selected, sum that score with cosine similarity to amplify concern sentiment
    if concern == 'sensitive_skin':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_3_score'],3)
#         prod2['new_sim'] = prod2['topic_3_score']
    elif concern == 'acne_control':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_4_score'],3)
#         prod2['new_sim'] = prod2['topic_4_score']
    elif concern == 'oil_control':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_5_score'],3)
#         prod2['new_sim'] = prod2['topic_5_score']
    elif concern == 'anti-aging':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_6_score'],3)
#         prod2['new_sim'] = prod2['topic_6_score']
    elif concern == 'none':
        prod2['new_sim'] = round(prod2['similarity'],3)
    
    # join other product info
    prod3 = pd.merge(prod2,product_list,left_on='other_asin',right_on='asin',how='inner')
    
    main_prod_info = product_list[product_list['asin']==asin]
    
    prods = prod3.sort_values(['category2','new_sim'],ascending=False)
#     recs=prods
    recs = prods.groupby(['category2']).head(5)
    
    main_prod.to_csv('results/'+asin+'_'+concern+'_main.csv', sep = ",")
    recs[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_3_score','topic_4_score','topic_5_score','topic_6_score']].to_csv('results/'+asin+'_'+concern+'_recs.csv', sep = ",")
    return main_prod_info, recs
#     return prod3

In [268]:
# test = get_rec('B0001EL5Q8') #anti-aging
# test = get_rec('B0001EL5JA') #hydrating
# main_prod, recs = get_rec('B0001EL5Q8','none')
# main_prod, recs = get_rec('B0001EL5Q8','anti-aging')
main_prod, recs = get_rec('B000UFWQJI','none')
main_prod2, recs2 = get_rec('B000UFWQJI','sensitive_skin')
print(main_prod.shape)
print(recs.shape)

(1, 8)
(35, 29)


In [194]:
main_prod

,id,asin,product,overall_rating,review_count,description,category2,category3
236,5dc881769f9b98109203c25a,B000UFWQJI,Epicuren Discovery Gelle Cleanser,4.5,11,"A deep cleansing, foaming gel that effectively...",Cleansers,Gels


#### check with no concerns

In [182]:
for _ in recs.loc[recs['other_asin']=='B000UFWQJI']['pos_sents']:
    print(_)

['this epicuren product works great.', '00. awesome dealarrived quickly, as described.', 'my husband loves this cleanser.', 'Great product from a reliable seller!Love this product.', 'Excellent cleanser!', 'I love Epicuren products, this gelle is perfect for combination skin and removes make up well.', 'Great value for a great productI love this cleanser so much!!!', 'I use it with my face brush and it cleans very well without feeling dry.', 'This cleanser was recommended to me for acne prone skin and it works well.', "It is nice and gentle and doesn't irritate my skinreceived in very well packed condition, love this brand, this is a go to for me."]


In [183]:
for _ in recs.loc[recs['other_asin']=='B000UFWQJI']['topic_3_sent']:
    print(_)

["It is nice and gentle and doesn't irritate my skinreceived in very well packed condition, love this brand, this is a go to for me."]


In [214]:
recs[['other_asin','product','category2','overall_rating','similarity','new_sim']]

,other_asin,product,category2,overall_rating,similarity,new_sim
0,B008H6MSZO,"Erno Laszlo Transphuse Age Defying Mask, 3.3 f...",Treatments & Masks,5.0,0.990278,0.990278
4,B0177RLDYU,"THALGO Mceutic Pro-Renewal Cleanser, 6.76 oz",Treatments & Masks,5.0,0.956414,0.956414
55,B00C1SZHBG,"DR. ALKAITIS Organic Flower Mask, 1.25 fl. oz.",Treatments & Masks,5.0,0.884563,0.884563
56,B00HYLOVLW,"Pevonia Micro Retinol Foaming Cleanser, 4 Fl Oz",Treatments & Masks,3.8,0.881868,0.881868
102,B00LNCB4A4,Borghese Fango Purificante Purifying Clay Clea...,Treatments & Masks,4.0,0.854337,0.854337
131,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.831555,0.831555
152,B01HBS7XP8,COSMEDIX Benefit Balance Antioxidant Infused T...,Toners & Astringents,5.0,0.811307,0.811307
164,B00T72SWZM,"COSMEDIX Purity Solution, 3.3 Fluid Ounce",Toners & Astringents,5.0,0.799324,0.799324
223,B002Q7K820,"Pevonia Spateen Blemished Skin Toner, 4 Fl Oz",Toners & Astringents,4.3,0.734536,0.734536
241,B0018QXRCI,"Elemis Cleanser, Rehydrating Ginseng Toner For...",Toners & Astringents,5.0,0.715760,0.715760


#### specify sensitive skin concern

In [211]:
recs2[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_3_score','topic_4_score','topic_5_score','topic_6_score']]

,other_asin,product,category2,overall_rating,similarity,new_sim,topic_3_score
179,B002RCQJ2W,Red Flower Lymphatic Phytopower Sea Cleanser a...,Treatments & Masks,4.4,0.788514,0.623399,0.790600
234,B00260GSDK,"DDF Glycolic 5% Exfoliating Wash, 8.45 fl. oz.",Treatments & Masks,3.1,0.719788,0.473260,0.657500
503,B011MFZM1I,IMAGE Skincare Ageless Total Resurfacing Masqu...,Treatments & Masks,4.2,0.505508,0.414567,0.820100
499,B00ZAA7O86,Laboratoires Filorga Hydra-Hyal Intensive Hydr...,Treatments & Masks,4.3,0.508875,0.411909,0.809450
540,B0057J2M9Q,"Natura Bisse Tolerance Recovery Cream, 1.7 fl....",Treatments & Masks,4.4,0.489045,0.397398,0.812600
223,B002Q7K820,"Pevonia Spateen Blemished Skin Toner, 4 Fl Oz",Toners & Astringents,4.3,0.734536,0.543777,0.740300
398,B00H59AA2W,"Guinot Hydra Beaute Toning Lotion, 6.7 Fl oz",Toners & Astringents,4.8,0.560834,0.510359,0.910000
131,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.831555,0.450675,0.541967
395,B00172DJ7Q,Mario Badescu Special Cleansing Lotion C,Toners & Astringents,3.8,0.562556,0.409693,0.728271
445,B000N2KJAG,Erno Laszlo Value Size Light Controlling Lotio...,Toners & Astringents,3.7,0.533711,0.365165,0.684200


#### compare top rated in that category for each

In [303]:
#no concerns - no sensitive skin references
samecat_none_asin = 'B000CR4GAQ'
for _ in recs.loc[recs['other_asin']==samecat_none_asin]['description']:
    print(_)

For combination, dry, mature and environmentally stressed skin types. June Jacobs, together with her daughter Rochelle, created this luxury skin care line centered on highly-efficacious, natural ingredients formulated to deliver immediate, impactful results with long-term benefits.


In [304]:
all_review_list[all_review_list['asin']==samecat_none_asin]['topic_3_sent']

100    []
Name: topic_3_sent, dtype: object

In [305]:
#sensitive skin - very positive sentiment
samecat_conc_asin = 'B002RCQJ2W'
for _ in recs2.loc[recs2['other_asin']==samecat_conc_asin]['description']:
    print(_)

transform the texture and tone of the skin through the regenerative powers of vitamin- rich marine plants, arctic white peat, organic mushrooms and potent helichrysum for a complete therapeutic experience. gently cleanse using a blend of aloe and sea algae that balances and repairs environmentally damaged skin. applied as a masque, it delivers a surge of moisture that replenishes the skin while reducing the appearance of fine lines. the whole essential oil of juniper berry gently plumps the complexion and conditions the skin, to reduce inflammation, puffiness and signs of acne and eczema. beta-glucans in certified organic reishi, chaga and maitake mushrooms encourage collagen production and rid the body of excess fluids and toxins. maitake mushrooms increase the recovery of stressed skin and stimulate circulation with a boost of hydration. one of the most powerful essential oils, helichrysum is known to heal and repair damaged skin, awakening a newfound radiance. red flower is a nyc ba

In [306]:
for _ in all_review_list[all_review_list['asin']==samecat_conc_asin]['topic_3_sent']:
    print(_)

['Ingredients are great, and it is safe to use on sensitive skin.']


#### compare top rated in other categories

In [319]:
#no concerns - mask
# diffcat_none_asin = 'B008H6MSZO' #mask
# diffcat_none_asin = 'B00260GSWQ' #exfoliator
diffcat_none_asin = 'B01HBS7XP8' #toner
for _ in recs.loc[recs['other_asin']==diffcat_none_asin]['description']:
    print(_)

Soothe, calm and nourish the skin with green tea, aloe vera and chamomile after a deep cleanse with Benefit Balance. Providing a second clean sweep of the skin with lactic acid and witch hazel, this antioxidant-infused toner helps protect the skin from harmful free radicals. D-glucuronic acid and shea butter help balance the skin and prevent moisture loss, making the skin perfectly prepped for serums or moisturizers. COSMEDIX skincare is formulated with the cleanest, most potent plant-based ingredients, Nobel Prize-winning chemistry and gentle actives to deliver safe, effective results. Trusted by dermatologists and aestheticians worldwide. With super clean ingredients, luxurious textures and pure botanicals, COSMEDIX gently delivers beautiful, transformative results for all skin types and tones. Our formulas feature higher concentrations of active ingredients and are created with a focus on crafting effective, gentle products that transform your skin naturally. Soothe, calm and nouris

In [320]:
for _ in all_review_list[all_review_list['asin']==diffcat_none_asin]['topic_3_sent']:
    print(_)

[]


In [321]:
#sensitive skin - 
# diffcat_conc_asin = 'B002RCQJ2W' #mask
# diffcat_conc_asin = 'B019QSK1YG' #exfoliator
diffcat_conc_asin = 'B002Q7K820' #toner
for _ in recs2.loc[recs2['other_asin']==diffcat_conc_asin]['description']:
    print(_)

Remove impurities and excess oil that can clog pores. Complete the cleanse, remove tap water impurities, control bacteria, soothe, heal and prepare the skin for further treatment with this alcohol-free toner. For teens and adults with acne and skin that tends to be sensitive or easily gets red and irritated. Favored by influencers, skincare experts and elite spas worldwide, award-winning Pevonia is clinically proven to work. Sustainably sourced, products are rich in potent organic ingredients for immediate and long-lasting results.


In [322]:
for _ in all_review_list[all_review_list['asin']==diffcat_conc_asin]['topic_3_sent']:
    print(_)

['It is a very gentle cleanser so i highly recommend it for those with sensitive skin.']


In [160]:
recs.iloc[0:5][['other_asin','product','topic_6_sent','topic_6_score','new_sim']]

,other_asin,product,topic_6_sent,topic_6_score,new_sim
35,B004WDLXI4,"St. Tropez Skin Illuminator, Gold, 1.6 fl. oz.",[Please give it try it will take your look nat...,0.9124,0.865542
111,B00LLJI36C,TNS Recovery Complex,[GreatJust re-purchased... Love this entire li...,0.9199,0.840827
62,B00DQYSXCY,AHAVA Dead Sea Osmoter Concentrate,[It's just helpful to have something to slow i...,0.8885,0.831469
106,B0032Z24M2,"Natura Bisse The Cure Pure Serum, 1.0 fl. oz.",[My skin has looked so good since switching to...,0.8788,0.804085
10,B000M4Z3OW,"NEOVA Serious Clarity Microderm Scrub, 2.5 Fl Oz",[Helps a great deal with fine lines.],0.8176,0.795044


In [152]:
all_review_list[all_review_list['asin']=='B005UPIVOC']

,asin,title,all_review_text,sentences,compound_scores,pos_sents,pos_scores,neg_sents,neg_scores,topic_3_sent,topic_3_score,topic_4_sent,topic_4_score,topic_5_sent,topic_5_score,topic_6_sent,topic_6_score
982,B005UPIVOC,Indie Lee Blemish Stick,As a mom and aesthetician I have tried endless...,[As a mom and aesthetician I have tried endles...,"[0.6696, 0.0, 0.6249, 0.0, 0.0, 0.0, 0.125, 0....",[As a mom and aesthetician I have tried endles...,"[0.6696, 0.6249, 0.125, 0.9532, 0.4389, 0.1779...","[Works overnight to reduce breakouts., At firs...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3089, -0.619...",[],NaN,[As a 40 year old woman who has battled acne s...,0.373067,"[Tea tree oil seemed to work better,]",0.4404,[:) My body naturally rejects anything that tr...,0.9532


In [151]:
for _ in all_review_list[all_review_list['asin']=='B005UPIVOC']['pos_sents']:
    print(_)

['As a mom and aesthetician I have tried endless products and this one is the best!', 'This is a great product.', "It's not as good as reviews I've read has said, but it's pretty good.", ":) My body naturally rejects anything that tries to help it, so I'm surprised this worked at least a little haha Amazing customer service and a great line of products :)Its literally alcohol, for the money use Q-tip and own alcohol save yourself few bucks.", 'As a 40 year old woman who has battled acne since I was a teenager, I think this stuff is a miracle!', 'It will easily dry out my skin now that Im older if I use more than just a small amount, but that is enough to dry up any blemishes.', 'My two teenage daughters who have my skin can tolerate more of it and it is very effective.', 'Perfect for suddenly appearing blemishes.', 'Works overnight!I really like this product.', '!I love this product so much!', 'I have tried virtually everything to help alleviate my breakouts: clay masks, charcoal masks

### repeat for acne example

In [252]:
main_prod, recs = get_rec('B019QSK1YG','none')
main_prod2, recs2 = get_rec('B019QSK1YG','acne_control')

In [234]:
main_prod[['asin','product','category2','overall_rating','description']]

,asin,product,category2,overall_rating,description
2073,B019QSK1YG,"Julep Korean Skincare Exfoliating, Hydrating F...","Exfoliators, Polishes & Scrubs",3.6,Get glowing in two easy steps: step 1: cleanse...


In [224]:
recs[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_4_score']]

,other_asin,product,category2,overall_rating,similarity,new_sim,topic_4_score
20,B00C1SZHBG,"DR. ALKAITIS Organic Flower Mask, 1.25 fl. oz.",Treatments & Masks,5.0,0.960862,0.960862,NaN
106,B0177RLDYU,"THALGO Mceutic Pro-Renewal Cleanser, 6.76 oz",Treatments & Masks,5.0,0.902600,0.902600,0.802000
184,B00HYLOVLW,"Pevonia Micro Retinol Foaming Cleanser, 4 Fl Oz",Treatments & Masks,3.8,0.852243,0.852243,0.648600
191,B00MOXQ8JI,Osmosis Skincare Replenish Antioxidant Repair ...,Treatments & Masks,3.3,0.848722,0.848722,NaN
198,B000CCML94,DERMAdoctor Wrinkle Revenge 1 antioxidant enha...,Treatments & Masks,4.3,0.836219,0.836219,0.480743
108,B01AHOFREW,jane iredale BeautyPrep Face Toner,Toners & Astringents,3.5,0.900087,0.900087,NaN
125,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.891415,0.891415,0.486833
131,B0018QXRCI,"Elemis Cleanser, Rehydrating Ginseng Toner For...",Toners & Astringents,5.0,0.889929,0.889929,NaN
225,B00T72SWZM,"COSMEDIX Purity Solution, 3.3 Fluid Ounce",Toners & Astringents,5.0,0.804593,0.804593,0.438000
232,B00H59AA2W,"Guinot Hydra Beaute Toning Lotion, 6.7 Fl oz",Toners & Astringents,4.8,0.796707,0.796707,NaN


In [225]:
recs2[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_4_score']]

,other_asin,product,category2,overall_rating,similarity,new_sim,topic_4_score
106,B0177RLDYU,"THALGO Mceutic Pro-Renewal Cleanser, 6.76 oz",Treatments & Masks,5.0,0.902600,0.723885,0.802000
184,B00HYLOVLW,"Pevonia Micro Retinol Foaming Cleanser, 4 Fl Oz",Treatments & Masks,3.8,0.852243,0.552765,0.648600
286,B001AIMZCW,"Elemis Maximum Replenish Night Cream, 1.7 Ounce",Treatments & Masks,5.0,0.699422,0.500436,0.715500
239,B00LNCB4A4,Borghese Fango Purificante Purifying Clay Clea...,Treatments & Masks,4.0,0.782944,0.406661,0.519400
459,B011MD169C,Replenix Pure Hydration Hyaluronic Acid Serum,Treatments & Masks,3.8,0.504699,0.404718,0.801900
125,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.891415,0.433971,0.486833
552,B00JJ4OCR0,Anthony Logistics for Men Purifying Astringent...,Toners & Astringents,4.0,0.459166,0.412285,0.897900
304,B002Q7K820,"Pevonia Spateen Blemished Skin Toner, 4 Fl Oz",Toners & Astringents,4.3,0.662625,0.398702,0.601700
349,B000RTTVGS,"Guinot Microbiotic Toning Lotion, 6.7 fl.oz.",Toners & Astringents,4.6,0.600675,0.375362,0.624900
225,B00T72SWZM,"COSMEDIX Purity Solution, 3.3 Fluid Ounce",Toners & Astringents,5.0,0.804593,0.352412,0.438000


#### compare top rated in that category for each

In [235]:
#no concerns - no acne references
for _ in recs.loc[recs['other_asin']=='B00260GSWQ']['topic_4_sent']:
    print(_)

['I had high hopes for this product since I have acne that forms deeply underneath my skin.', "I was hoping this product would help soothe those out or prevent further breakouts but eh it doesn't really do anything.", 'I would return this but I already threw away the packaging box... maybe I still can?After having acne prone skin for 10+ years, and wanting to avoid any drastic medications, DDF was my saving grace.']


In [236]:
for _ in recs.loc[recs['other_asin']=='B00260GSWQ']['pos_sents']:
    print(_)

['I had high hopes for this product since I have acne that forms deeply underneath my skin.', "They almost seem like bumps and I can't pop them.", "I was hoping this product would help soothe those out or prevent further breakouts but eh it doesn't really do anything.", 'I would return this but I already threw away the packaging box... maybe I still can?After having acne prone skin for 10+ years, and wanting to avoid any drastic medications, DDF was my saving grace.', "com and have it delivered to a UPS store by the border (I live in Canada and it's not available for a reasonable price at Amazon.", "ca), it's completely worth the drive!This is the best exfoliation cleanser I have ever used.", '!This is the very best exfoliation cleanser I have ever used and believe me I have used "many"!', 'I like the fact that there are no artificial colors or fragrances with natural ingredients.', 'It works very well with the DDF cleansing system and I use it 2 times a week.', 'I am so happy that I c

In [237]:
#acne - very positive sentiment
for _ in all_review_list[all_review_list['asin']=='B00070JJ4G']['topic_4_sent']:
    print(_)

['I have very sensitive skin, but have no problems with this product and my skin is left soothed and Smooth!I use most of the Neova products with great success - so it came as no surprise that I LOVE the moisture mask.']


#### compare top rated in other categories

In [238]:
#no concerns - treatment and masks
for _ in all_review_list[all_review_list['asin']=='B01AHOFREW']['topic_4_sent']:
    print(_)

[]


In [239]:
for _ in recs.loc[recs['other_asin']=='B01AHOFREW']['pos_sents']:
    print(_)

["the consistency is so thin that it's like I'm using waterSpectacular productsI have only used the three step system for close to a week but I can tell my face is healing quickly and I have had a couple of comments on my skin looking radiant!I love jane iredale makeup but not wild about the cleansing systemthe product was only half full and that makes me worry if it was resold to me"]


In [242]:
#acne - treatment and masks B0019V5HI4
for _ in all_review_list[all_review_list['asin']=='B01CQ4862G']['topic_4_sent']:
    print(_)

["My esthetician recommended it to me for a non ashen look that most leave, you won't have a problem with this one!Great product and service at a good value."]


In [ ]:
# info

In [244]:
review_list.shape

(81038, 5)

In [176]:

# for prod in product_list['description']:
#     if 'sensit' in prod:
#         print(prod)

In [221]:
a = max(all_review_list['topic_4_score'])

In [222]:
all_review_list.loc[all_review_list['topic_4_score']==a]

,asin,title,all_review_text,sentences,compound_scores,pos_sents,pos_scores,neg_sents,neg_scores,topic_3_sent,topic_3_score,topic_4_sent,topic_4_score,topic_5_sent,topic_5_score,topic_6_sent,topic_6_score
2073,B019QSK1YG,"Julep Korean Skincare Exfoliating, Hydrating F...",Can really see the difference with my skin... ...,[Can really see the difference with my skin......,"[0.0, 0.0, 0.6817, 0.1779, 0.128, 0.0, 0.0, 0....","[One of my FAVORITE Julep products!!!, This is...","[0.6817, 0.1779, 0.128, 0.128, 0.7959, 0.4574,...",[Can really see the difference with my skin......,"[0.0, 0.0, 0.0, 0.0, -0.2057, 0.0, -0.8251, 0....",[Juelp provides the best skin care regimen for...,0.88355,"[I love it, I love the Kojac sponge effective ...",0.9545,[This cleansing oil leaves my skin feeling so ...,0.580058,[],NaN


In [259]:


for _ in all_review_list[all_review_list['asin']=='B002DP8D36']['sentences']:
    print(_)

['I first tried this kit about a year ago.', 'I loved every product in it, but really liked the face wash and the peel.', "I've tried Ole H. peel and it is nowhere near as effective as this one.", 'In fact, it always makes my skin break out.', "But I've had NO clogged pores or breakouts with this set.", "I was beginning to get age spots, one on each side of my face where I guess I didn't put enough sunscreen over the years.", 'I found that when I used these products, the spots fade.', "When I stop using them (and I'm terrible about using the sunscreen so it's not just that), the spots return.", 'I have more glow-y skin than I have had in years.', 'Highly recommended!', '!This set was a gift from my daughter and I loved every product.', 'I have dry aging skin and the products were wonderful.', 'The cleanser tightened and cleansed beautifully.', 'The serum is light and very nourishing.', 'The Moisturizer contains more peptides than the $100 stuff I buy from my dermatologists office and f

In [278]:
product_list[product_list['asin']=='B00K1LYKUO']

,id,asin,product,overall_rating,review_count,description,category2,category3
1569,5dc881789f9b98109203dcc1,B00K1LYKUO,IMAGE Skincare The Max Stem Cell Facial Cleans...,4.4,82,This luxurious cleanser is powered by advanced...,Cleansers,Not listed


In [ ]:
# Save out dictionaries for flask app
# dictionary of product info - asin, product name, description, overall rating, category, topic values?
# dictionary of similarities (sim2)